In [167]:
import pandas as pd

In [168]:
df = pd.read_csv('nluk2017053w.csv')

In [169]:
df.head()

,c,Lat,Lon
0,1,60.200835,11.097693
1,1,60.194509,11.098558
2,1,60.192456,11.095304
3,1,59.328008,18.091634
4,1,56.971745,24.160066


In [170]:
len(df)

973401

In [171]:
ndf = df.sample(90000)

In [172]:
ndf.head()

,c,Lat,Lon
554894,1,52.087386,5.245283
439656,1,52.303727,4.691178
631046,1,52.040996,4.368563
886726,1,51.571275,4.634862
834032,1,51.825215,5.801308


In [173]:
rdf = ndf.drop(columns=['c'])

In [174]:
rdf.describe()

,Lat,Lon
count,90000.000000,90000.000000
mean,52.143746,4.898846
std,0.406108,0.695765
min,35.912115,0.071257
25%,51.918004,4.494245
50%,52.198778,4.865970
75%,52.363824,5.118975
max,55.628446,33.215646


In [175]:
rdf[rdf.Lon > 7]

,Lat,Lon
973308,45.486987,9.157694
973378,37.190322,33.215646
973257,50.363218,7.609217
34407,53.014155,8.757048
973310,45.471460,9.184316
973280,47.686280,11.757834
973321,43.858746,18.422192
10,55.628446,12.649165
973345,40.392682,17.453066
973315,44.530673,11.287344


In [176]:
rdf=rdf[rdf.Lon < 7]

In [177]:
rdf.describe()

,Lat,Lon
count,89984.000000,89984.000000
mean,52.144716,4.897054
std,0.391865,0.676027
min,43.594060,0.071257
25%,51.918022,4.494235
50%,52.199292,4.865941
75%,52.363824,5.118625
max,53.420130,6.932523


In [178]:
from sklearn.cluster import KMeans

In [179]:
kmeans = KMeans(n_clusters = 25).fit(rdf)

In [180]:
clusters=pd.DataFrame(kmeans.cluster_centers_, columns=['lat', 'lon'])

In [181]:
clusters.head()

,lat,lon
0,52.360276,4.885882
1,51.904673,5.910257
2,52.072676,4.305940
3,53.189430,6.567475
4,51.238636,0.461222


In [182]:
clusters.lat.values

array([ 52.36027631,  51.90467333,  52.07267639,  53.18942969,
        51.23863593,  51.7151807 ,  51.91068079,  53.12284256,
        50.90476829,  52.25701154,  51.52342765,  52.30013368,
        52.69043283,  51.63057987,  52.54898578,  51.44904895,
        52.3697336 ,  53.03614791,  52.16259396,  52.1330583 ,
        51.43900548,  52.71239881,  52.58291835,  52.17995454,  52.07854755])

In [183]:
from bokeh.io import output_notebook, show
from bokeh.models import (GMapOptions, ColumnDataSource)
from bokeh.plotting import gmap
map_options = GMapOptions(lat=52.355, lng=5, map_type="roadmap", zoom=14)

GOOGLE_API_KEY = ''

with open('google_api.key', 'r') as myfile:
    GOOGLE_API_KEY = myfile.read()

plot = gmap(GOOGLE_API_KEY, map_options, title="Amsterdam")

source = ColumnDataSource(
    data=dict(
        lat=clusters.lat.values,
        lon=clusters.lon.values)
)

plot.circle(x="lon", y="lat", size=15, fill_color='blue', fill_alpha=0.8, source=source)
output_notebook()

show(plot)

Loading BokehJS ...

In [184]:
# lets filter out everything but IJburg PAs
ijdf = df[df.Lat < 52.362]
ijdf = ijdf[ijdf.Lat > 52.345]
ijdf = ijdf[ijdf.Lon < 5.0170]
ijdf = ijdf[ijdf.Lon > 4.9850]

len(ijdf)

6661

In [185]:
map_options = GMapOptions(lat=52.355, lng=5, map_type="roadmap", zoom=14)
plot = gmap(GOOGLE_API_KEY, map_options, title="IJburg")

source = ColumnDataSource(
    data=dict(
        lat=ijdf.Lat.values,
        lon=ijdf.Lon.values)
)

plot.circle(x="lon", y="lat", size=1, fill_color='blue', fill_alpha=0.8, source=source)
output_notebook()

show(plot)

Loading BokehJS ...

In [186]:
class CoordinatesConvertor(object):
    def __init__(self, minLon, minLat, width = 0.001, height = 0.0005):
        ''' Maps lat lon to a cell.
            Keyword arguments:
            minLon -- min lon in data set
            minLat -- min lat in data set
            width -- the width of a cell in fraction of lon (default 0.001)
            height -- the heigth of a cell in fraction of lat (default 0.0005)
        '''
        self._minLon = minLon
        self._minLat = minLat
        self._cell_width_lon = width
        self._cell_height_lat = height
        
    def toX(self, lon):
        return int((lon - self._minLon) / self._cell_width_lon)
    
    def toY(self, lat):
        return int((lat - self._minLat) / self._cell_height_lat)

In [187]:
#convertor = CoordinatesConvertor(ijdf.Lon.min(), ijdf.Lat.min())
#ijdf['x'] = ijdf.apply(lambda row: convertor.toX(row['Lon']), axis = 1)
#ijdf['y'] = ijdf.apply(lambda row: convertor.toY(row['Lat']), axis = 1)

In [188]:
class CoordinatesMapper(object):
    def __init__(self, minLon, minLat, maxLon, maxLat, hCount = 11, vCount = 11):
        ''' Maps lat lon to a cell.
            Keyword arguments:
            minLon -- min lon in data set
            minLat -- min lat in data set
            maxLon -- max lon in data set
            maxLat -- max lat in data set
            hCount -- amount of areas horisontally
            vCount -- amount of areas vertically
        '''
        self._minLon = minLon
        self._minLat = minLat
        self._maxLon = maxLon
        self._maxLat = maxLat
        self._hCount = hCount
        self._vCount = vCount
        self._cell_width_lon = round((self._maxLon - self._minLon) / self._hCount,3)
        self._cell_height_lat = round((self._maxLat - self._minLat) / self._vCount,3)
        #print (self._minLat)
        #y = int(round(self._maxLat - self._minLat,3) / self._cell_height_lat) * hCount
        #print (y)
        #print (self._cell_width_lon)
        #print (self._cell_height_lat)
        
    def toIndex(self, lon, lat):
        
        
        y = int(round(lat - self._minLat,3) / self._cell_height_lat)
        x = int((lon - self._minLon) / self._cell_width_lon)
        
        return self._hCount * y + x

In [189]:
mapper = CoordinatesMapper(ijdf.Lon.min(), ijdf.Lat.min(), ijdf.Lon.max(), ijdf.Lat.max())
ijdf['i'] = ijdf.apply(lambda row: mapper.toIndex(row['Lon'] ,row['Lat']), axis = 1)

ijdf.describe()

,c,Lat,Lon,i
count,6661.000000,6661.000000,6661.000000,6661.000000
mean,2.042336,52.354575,4.999312,104.987990
std,12.179267,0.003661,0.007845,38.241765
min,1.000000,52.345396,4.985412,1.000000
25%,1.000000,52.351955,4.991944,82.000000
50%,1.000000,52.354534,4.998222,104.000000
75%,1.000000,52.358178,5.006592,144.000000
max,504.000000,52.361880,5.016977,178.000000


In [190]:
map_options = GMapOptions(lat=52.355, lng=5, map_type="roadmap", zoom=15)
plot = gmap(GOOGLE_API_KEY, map_options, title="IJburg")

colors = ["#ff0000", "#00ff00", "#0000ff, #ffff00", "#00ffff", "#ff00ff"]
points_colors = ijdf.apply(lambda row: colors[int(row['i'] % len(colors))] , axis = 1).values

source = ColumnDataSource(
    data=dict(
        lat=ijdf.Lat.values,
        lon=ijdf.Lon.values,
        color = points_colors
    )
)

plot.circle(x="lon", y="lat", size=2, fill_color='color', fill_alpha=1, source=source)
output_notebook()

show(plot)

Loading BokehJS ...